In [1]:
import json
import sys
import os
import numpy as np
from urllib import request
import requests

## Test what data do we have

In [2]:
def parse_info(url,file_name):    
#     response = requests.get(url)
#     with open(file_name, "wb") as f:
#         f.write(response.read())
        
    head = {}
    head['User-Agent'] = 'Mozilla/5.0 (Linux; Android 4.1.1; Nexus 7 Build/JRO03D) AppleWebKit/535.19 (KHTML, like Gecko) Chrome/18.0.1025.166  Safari/535.19'
    req = request.Request(url, headers=head)
    response = request.urlopen(req)
    with open(file_name, "wb") as f:
        f.write(response.read())

In [3]:
# listen record
with open('./testData/myRecord.json') as record:
    record_json = json.load(record)['allData']
    
#friends id, top100 music input userid    
# follow list
with open('./testData/myFollows.json') as follows:
    follows_json = json.load(follows)['follow']
    
with open('./testData/detail.json') as user:
    user_json = json.load(user)['profile']

In [4]:
class User():
    def __init__(self, user_id, user_name, followeds, follows, playlistCount):
        self.user_id = user_id
        self.user_name = user_name
        self.followed_num = followeds
        self.follows_num = follows
        self.playlist_num = playlistCount
        self.follows_list = []
    
    def getUserId(self):
        return self.user_id
    
    def getUserName(self):
        return self.user_name
    
    def getFollowedNum(self):
        return self.followed_num
    
    def getFollowsNum(self):
        return self.follows_num
    
    def getPlaylistNum(self):
        return self.playlist_num
    
    def getFollowsList(self):
        return self.follows_list
    
    def setUserId(self, user_id):
        self.user_id = user_id
    
    def setUserName(self, user_name):
        self.user_name = user_name
    
    def setFollowedNum(self, followed_num):
        self.followed_num = followed_num
    
    def setFollowsNum(self, follows_num):
        self.follows_num = follows_num
    
    def setPlaylistNum(self, playlist_num):
        self.playlist_num = playlist_num
        
    def setFollowsList(self, follows_list):
        self.follows_list = follows_list

In [15]:
class Music():
    def __init__(self, song_id, song_name, score):
        self.song_id = song_id
        self.song_name = song_name
        self.score = score
    
    def getSongId(self):
        return self.song_id
    
    def getSongName(self):
        return self.song_name
    
    def getScore(self):
        return self.score
    
    def setSongId(self, song_id): 
        self.song_id = song_id
        
    def setSongName(self, song_name):
        self.song_name = song_name
        
    def setScore(self, score):
        self.score = score

In [16]:
# get a user's follows list
# input: uId
# output: followsId(list of ids), userList(List of User objs) 
def getFollows(uId):
    followsId = []
    userList = []
    filepath = './trainData/FollowsInfo/'+uId+'_follows.json'
    if(not os.path.exists(filepath)):
        parse_info('http://localhost:3000/user/follows?uid='+uId, filepath)
    with open(filepath) as follows:
        follows_json = json.load(follows)['follow']
    for i in range (len(follows_json)):
        user_id = str(follows_json[i]['userId'])
        user_name = follows_json[i]['nickname']
        followeds = follows_json[i]['followeds']
        follows = follows_json[i]['follows']
        playlistCount = follows_json[i]['playlistCount']
        followsId.append(user_id)
        userList.append(User(user_id, user_name, followeds, follows, playlistCount))
        
    return followsId, userList


# get a user's record list
# input: uId
# output: songList(list of Music objs)
def getRecord(uId):
    songList = []
    filepath = './trainData/MusicInfo/'+uId+'_record.json'
    if(not os.path.exists(filepath)):
        parse_info('http://localhost:3000/user/record?uid='+uId+'&type=0',filepath)
    with open(filepath) as records:
        records = json.load(records)['allData']
    for i in range(len(records)):
        song_id = records[i]['song']['id']
        song_name = records[i]['song']['name']
        score = records[i]['score']
        songList.append(Music(song_id, song_name, score))
    return songList

# get a user's profile
# input: uId
# output: user
def getUserInfo(uId):
    filepath = './trainData/UserInfo/'+uId+'_user.json'
    if(not os.path.exists(filepath)):
        parse_info('http://localhost:3000/user/detail?uid='+uId, filepath)
    with open(filepath) as userInfo:
        user_json = json.load(userInfo)['profile']
    
    user_id = user_json['userId']
    user_name = user_json['nickname']
    followeds = user_json['followeds']
    follows = user_json['follows']
    playlistCount = user_json['playlistCount']
    
    user = User(user_id, user_name, followeds, follows, playlistCount)
        
    return user
    
followsId, userList = getFollows('40356946')
# songList = getRecord('40356946')
# user_me = getUserInfo('40356946')

## Build co-occurrence matrix

### get users' info

In [17]:
import queue
# start from one user, find a certain number of users, find their record, filter out those with pop less than 10.
start_user_id = '40356946'
start_user = getUserInfo(start_user_id)
q = queue.Queue()

q.put(start_user_id)

user_dict = {start_user_id:start_user}

count = 0
while(len(user_dict) < 50000):
    cur_user_id = q.get()
    cur_user = user_dict[cur_user_id]
#     print(cur_user_id)
    try:
        followsId, userList = getFollows(cur_user_id)
    except ValueError:
        print('value error in this user\'s file!')
        continue
    
    cur_user.setFollowsList(followsId)
    
    count += 1
    if(count % 500 == 0):
        print('%d th query, %d users fetched'%(count, len(user_dict)))
    for (followsId, user) in zip(followsId, userList):
        if(followsId not in user_dict):
            user_dict[followsId] = user
            q.put(followsId)

500 th query, 5281 users fetched
value error in this user's file!
1000 th query, 10789 users fetched
1500 th query, 15253 users fetched
value error in this user's file!
value error in this user's file!
value error in this user's file!
value error in this user's file!
value error in this user's file!
value error in this user's file!
value error in this user's file!
value error in this user's file!
value error in this user's file!
value error in this user's file!
value error in this user's file!
value error in this user's file!
value error in this user's file!
value error in this user's file!
value error in this user's file!
value error in this user's file!
value error in this user's file!
value error in this user's file!
value error in this user's file!
value error in this user's file!
value error in this user's file!
value error in this user's file!
value error in this user's file!
value error in this user's file!
value error in this user's file!
value error in this user's file!
value 

In [18]:
#user_dict: uid -> user object
songList = getRecord('40356946')

In [26]:
# merge all songlists
songList

In [ ]:
# get co-occurrence